## Setup

Requirements:

- Python 3.8
- CPLEX Studio 20.1.0
- docplex library configured - only works with Python 3.8

## Import dependencies

In [1]:
# namedtuple creates a tuple with a name
# https://docs.python.org/3/library/collections.html#collections.namedtuple
from collections import namedtuple

# Model: https://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.model.html
# Model is a class to embed modeling objects
from docplex.mp.model import Model
# LinearExpr contains the negate() function to create negate a linear expression
from docplex.mp.linear import LinearExpr

## Initialise Data

In [2]:
# Initialise data
Unit = namedtuple('Unit', ['name', 'att_strength', 'recruit_time_in_seconds', 'food'])
units = [
    Unit('axe', 45, 90, 1),
    Unit('lc', 130, 360, 4),
    Unit('ma', 150, 450, 5),
    Unit('serk', 300, 1200, 6),
    Unit('ram', 2, 480, 5),
]

## Start a new model

In [3]:
# Alternatively, I can write
# with Model() as mdl:
#   ... # do things here
m = Model(name = 'Max strength unit time')

### Decision variables

In [4]:
number_of_units = m.integer_var_dict(
    [u.name for u in units], name = 'number_of_units'
)
number_of_units

{'axe': docplex.mp.Var(type=I,name='number_of_units_axe'),
 'lc': docplex.mp.Var(type=I,name='number_of_units_lc'),
 'ma': docplex.mp.Var(type=I,name='number_of_units_ma'),
 'serk': docplex.mp.Var(type=I,name='number_of_units_serk'),
 'ram': docplex.mp.Var(type=I,name='number_of_units_ram')}

### Decision expressions

In [5]:
m.totalBuildTime = m.sum([number_of_units[u.name] * u.recruit_time_in_seconds for u in units])
m.totalBuildTime

docplex.mp.LinearExpr(90number_of_units_axe+360number_of_units_lc+450number_of_units_ma+1200number_of_units_serk+480number_of_units_ram)

In [6]:
m.totalNegativeAttack = LinearExpr.negate(m.sum([number_of_units[u.name] * u.att_strength for u in units]))
m.totalNegativeAttack

docplex.mp.LinearExpr(-45number_of_units_axe-130number_of_units_lc-150number_of_units_ma-300number_of_units_serk-2number_of_units_ram)

In [7]:
m.totalFood = m.sum([number_of_units[u.name] * u.food for u in units])
m.totalFood

docplex.mp.LinearExpr(number_of_units_axe+4number_of_units_lc+5number_of_units_ma+6number_of_units_serk+5number_of_units_ram)

### Constraints

From an error I got when trying to add constraints: `Only <=, ==, >= are allowed`

In [8]:
ct_food_no_church = m.add_constraints([m.totalFood <= 20596, m.totalFood >= 20580])
ct_food_no_church

[docplex.mp.LinearConstraint[](number_of_units_axe+4number_of_units_lc+5number_of_units_ma+6number_of_units_serk+5number_of_units_ram,LE,20596),
 docplex.mp.LinearConstraint[](number_of_units_axe+4number_of_units_lc+5number_of_units_ma+6number_of_units_serk+5number_of_units_ram,GE,20580)]

In [9]:
ct_must_have_rams = m.add_constraint(number_of_units['ram'] >= 250)
ct_must_have_rams

docplex.mp.LinearConstraint[](number_of_units_ram,GE,250)

In [10]:
# Need to modify our 0 < totalBuildTime <= 4 * 7 * 24 * 3600 to 1 <= totalBuildTime <= 4 * 7 * 24 * 3600 due to docplex limitation
ct_build_time_less_than_4_weeks = m.add_constraints([1 <= m.totalBuildTime, (m.totalBuildTime <= 4 * 7 * 24 * 3600)])
ct_build_time_less_than_4_weeks

[docplex.mp.LinearConstraint[](90number_of_units_axe+360number_of_units_lc+450number_of_units_ma+1200number_of_units_serk+480number_of_units_ram,GE,1),
 docplex.mp.LinearConstraint[](90number_of_units_axe+360number_of_units_lc+450number_of_units_ma+1200number_of_units_serk+480number_of_units_ram,LE,2419200)]

### Objectives

In [11]:
m.add_kpi(m.totalNegativeAttack, "Total negative attack strength")
m.add_kpi(m.totalBuildTime, "Total build time")

DecisionKPI(name=Total build time,expr=90number_of_units_axe+360number_of_units_lc+450number_of_units_m..)

### Solve

It doesn't look like docplex.mp supports staticLexFull.

In [12]:
m.minimize_static_lex([m.totalNegativeAttack, m.totalBuildTime])
m.solve()

docplex.mp.solution.SolveSolution(obj=-896420,values={number_of_units_ax..

### Print the results

In [13]:
if m.get_solve_status() == None:
    print("Model is infeasible")
else:
    print(
        f"axe: {number_of_units['axe'].solution_value}"
        f", lc: {number_of_units['lc'].solution_value}"
        f", ma: {number_of_units['ma'].solution_value}"
        f", serk: {number_of_units['serk'].solution_value}"
        f", ram: {number_of_units['ram'].solution_value}"
        f", food: {sum([number_of_units[u.name].solution_value * u.food for u in units])}"
        f", time: {round(sum([number_of_units[u.name].solution_value * u.recruit_time_in_seconds for u in units]) / (24 * 3600), 2)} days"
    )

axe: 14276.0, lc: 0, ma: 0, serk: 845.0, ram: 250.0, food: 20596.0, time: 28.0 days
